### imports

In [1]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### git and github setup

In [5]:
from os.path import join
from google.colab import drive
drive.mount('/content/drive')

ROOT = '/content/drive/My Drive/'
with open(join(ROOT, "authorize.txt"), "r") as textfile:
    lines = textfile.readlines()
GIT_USERNAME = lines[0]
GIT_TOKEN = lines[2]
GIT_REPOSITORY = "eeg-classification" #@param {type:"string"}
MY_GOOGLE_DRIVE_PATH = GIT_REPOSITORY #@param {type:"string"}
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"

GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"

CLONE_IT = False #@param {type:"boolean"}
if CLONE_IT:
  !git clone "{GIT_PATH}" "{PROJECT_PATH}"
os.chdir(PROJECT_PATH)
!git checkout dev || git checkout -b dev
!git config --global user.email lines[1]
!git push --set-upstream origin dev

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
PROJECT_PATH:  /content/drive/My Drive/eeg-classification
mkdir: cannot create directory ‘/content/drive/My Drive/eeg-classification’: File exists
Already on 'dev'
Your branch is up to date with 'origin/dev'.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Everything up-to-date


### load the data

In [36]:
folderPath = pathlib.Path("/content/drive/My Drive/data/EEG_DATA/")

_X_train = np.load(folderPath/'X_train_valid.npy')
_X_test = np.load(folderPath/'X_test.npy')
_y_train = np.load(folderPath/'y_train_valid.npy')
_y_test = np.load(folderPath/'y_test.npy')

In [37]:
_X_train.shape, _X_test.shape, _y_train.shape, _y_test.shape

((2115, 25, 1000), (443, 25, 1000), (2115,), (443,))

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid  = train_test_split(
    _X_train, _y_train, test_size=0.1, random_state=42)

In [45]:
X_test = _X_test
y_test = _y_test

In [46]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
print(X_valid.shape, y_valid.shape)

(1903, 25, 1000) (1903,)
(443, 25, 1000) (443,)
(212, 25, 1000) (212,)


### conv net